In [84]:
import numpy as np
import scipy.linalg as scipy
import matplotlib.pyplot as plt

In [85]:
def new_rand_A(size_n, size_m):
    A = np.zeros((size_n,size_m))
    for i in range(size_n):
        for j in range(size_m):
            A[i][j] = np.random.randint(low=0, high=9)
    
    return A

A_10 = new_rand_A(10,8)
A_100 = new_rand_A(100,100)

def secretly_rank_k(size_n, size_k):
    a = np.zeros((size_n,size_k))
    b = np.zeros((size_k, size_n))
    
    for i in range(size_n):
        for j in range(size_k):
            a[i][j] = np.random.randint(low=-9, high=9)
            b[j][i] = np.random.randint(low=-9, high=9)
    
    part = int(size_n/2)
    choice = np.random.choice(range(1,part))
    return np.matmul(a, b)[:,choice:]

def pivot_A(A, p):
    new_A = np.zeros((len(A),len(A[0])))
    for col in range(len(p)):
        new_A[:,col] = A[:,p[col]]
    return new_A

def Swap(arr, j, max_index):
    #arr[:, [start_index, last_index]] = arr[:, [last_index, start_index]]
    j_col = arr[:][j]
    max_col = arr[:][max_index]
    newarr = np.copy(arr)
    newarr[:][j] = j_col
    newarr[:][max_index] = max_col
    
    return newarr

S_100 = secretly_rank_k(100, 3)
Q,R,p = scipy.qr(A_10, pivoting=True)
u,s,v = np.linalg.svd(S_100)


In [86]:
def apx_rank_k(size_n, rank, TOL_ord):
    a = np.zeros((size_n,size_n))
    b = np.zeros((size_n, size_n))
    for i in range(size_n):
        for j in range(size_n):
            a[i][j] = np.random.randint(low=-9, high=9)
            b[j][i] = np.random.randint(low=-9, high=9)
            
    U,R = scipy.qr(a)
    V,R = scipy.qr(b)
    #TOL_ord = TOL_ord - 2
    rang = np.linspace(rank-TOL_ord,-size_n + rank - TOL_ord, size_n)
    singular_vals = []
    for i in rang:
        singular_vals.append(10**i)
    for i in range(len(singular_vals)):
        #print("rank:", i+1, "value:", singular_vals[i])
        None
    s = np.diag(singular_vals)
    matrix = np.matmul(U,s,V)
    return matrix

In [87]:
A = new_rand_A(100,95)
S = secretly_rank_k(100, 10)
A_2 = np.zeros((3,3))
ev = 2
for i in range(2):
    for j in range(3):
        A_2[i][j] = ev
        ev = ev*2
A_2[2,0] = 1
A_2[2,1] = 2
A_2[2,2] = 3

In [88]:
def RRQR_1(A, TOL=None):
    Q,R,p = scipy.qr(A, pivoting=True)          #get qr decomposition with pivoting from scipy
    pivoted_A = pivot_A(A,p)                    #pivot columns of A based on QR decomp
    k = 1                                       #starting rank = 1
    r_1 = R[0,:]                                #starting slices of the QR decomp, not important just for reference
    r_11 = R[0,0]
    r_22 = R[1:,1:]
    q_1 = Q[:,0]
    q_2 = Q[:,1:]
    
    if TOL == None:                             #default tolerance if none is given
        TOL = 10e-5
    
    err = np.linalg.norm(r_22, ord=2)           #error is equal to norm of the part of the approximated matrix that is not used
    while err > TOL and k<min(len(A), len(A[0])):
        #print("current rank, error",k, err)                              #while error is greater than tolerance, increase
        k += 1                                  #increase rank
        
        r_1 = R[:k,:]                           #update slices
        r_11 = R[:k,:k]
        r_22 = R[k:,k:]
        q_1 = Q[:,:k]
        q_2 = Q[:,k:]
        
        if len(r_11[0]) == len(R[0]):           #if approximate array is max length of R, can't update error
            None
        else:
            err = np.linalg.norm(r_22, ord=2)
        
    #print("final error: ",np.linalg.norm(r_22, ord=2))
    
    return k

k = RRQR_1(A_2, None)
print("estimated rank",k)
print("actual rank", np.linalg.matrix_rank(A_2))

k = RRQR_1(S, None)
print("estimated rank",k)
print("actual rank", np.linalg.matrix_rank(S))
'''Q,R,k = RRQR(S_100, None)
print(k)'''

estimated rank 2
actual rank 2
estimated rank 10
actual rank 10


'Q,R,k = RRQR(S_100, None)\nprint(k)'

In [89]:
'''TEST CELL FOR ABOVE'''
'''TEST FOR EXACTLY RANK K'''
'''counter = 0
for i in range(1000):
    counter = 0
    random_m = np.random.choice(range(50,100))
    random_k = np.random.choice(range(1,50))
    S = secretly_rank_k(random_m, random_k)
    rrqr = RRQR_1(S, TOL = 10e-10)
    svd = np.linalg.matrix_rank(S)
    #print(rrqr, svd)
    if rrqr != svd:
        print("wrong!")
        counter += 1
        
print("accuracy:", (1000-counter)/1000)
'''
'''TEST FOR APPROXIMATELY RANK K'''

'''counter = 0
for i in range(1000):
    random_m = np.random.choice(range(50,100))
    random_k = np.random.choice(range(1,50))
    apx = apx_rank_k(random_m, random_k, 10)
    res = RRQR_1(apx, TOL = 10e-10)
    counter += abs(random_k - res)
    if i%100 == 0:
        print(i)
print("avg difference", counter/1000)'''

rank, arr = RRQR_1(S,10e-10)
plt.plot(range(len(arr)), arr)

TypeError: cannot unpack non-iterable int object

In [90]:
'''RRQR with column pivoting using householder'''
'''FAIL'''
#https://codereview.stackexchange.com/questions/171503/implementation-of-the-householder-transformation
def householder(A):
    (n,m)=A.shape
    p=min(n,m)
    alpha=np.zeros(m)
    for j in range(0,p):
        alpha[j]=np.linalg.norm(A[j:,j])*np.sign(A[j,j])
        if (alpha[j]!=0):
            beta=1/np.sqrt(2*alpha[j]*(alpha[j]+A[j,j]))
            A[j,j]=beta*(A[j,j]+alpha[j])
            A[j+1:,j]=beta*A[j+1:,j]
            for k in range(j+1,m):
                gamma=2*A[j:,j].dot(A[j:,k])
                A[j:,k]=A[j:,k]-gamma*A[j:,j]
    return A,alpha

def RRQR_2(A, TOL=None):
    m = len(A)                  #number of rows
    n = len(A[0])               #number of cols
    colnrm = np.zeros(n)
    for j in range(n):          #insert values into column norm vector
        colnrm[j] = np.linalg.norm(A[:,j], ord=2)
    
    max_col = max(colnrm)
    max_col_index = np.argmax(colnrm)
    
    if TOL == None:                             #arbitrary tolerance if none is given
        TOL = 10e-5
    
    j = 0
    while j < n and max_col > TOL:
        A = Swap(A,j,max_col_index)
        colnrm = Swap(colnrm, j, max_col_index)
        
        A[j:m, j+1:n] = householder(A[j:m, j+1:n])[0]
        colnrm[j+1:n] -= (A[j, j+1:n])**2
        
        max_col = max(abs(colnrm[j:]))
        max_col_index = np.argmax(colnrm)
        j += 1
    return j
A = apx_rank_k(30,7, 10)
RRQR_2(S, 10e-10)

93

In [91]:
'''interpolative decomposition whee'''

def interdecomp(A, k):
    Q,R = np.linalg.qr(A); '''we want to start with QR '''
    '''separating our R matrix into R11 and R12; we are disregarding the R22 this time around because its not actually part of the factorization'''
    R11 = R[:k,:k];
    R12 = R[:k,(k+1):]; 
    A1 = A[:,:k] 
    R11inverse  = np.linalg.inv(R11);
    T = np.dot(R11inverse, R12);
    Ik = np.identity(k);
    return A1, Ik, T;      

'''we still have k rows, but we want to have k+1 to the end columns'''
'''now our T is equal to R11^-1 times R12 and our I is just an identity matrix of size k and A1 is the first k columns of our matrix A'''
'''interpolative decomp = A1 * I_k + T'''
'''all the rows but k columns'''

'all the rows but k columns'

In [92]:
first_test = apx_rank_k(10,4,10)
rank = RRQR_1(first_test, 10e-10)
print("rank given by algorithm", rank)
A1, Ik, T = interdecomp(first_test, rank)
z_col = np.zeros((3,1))
b = np.hstack((Ik,T,z_col))
full_ID = np.dot(A1,b)
print(np.linalg.norm(first_test - full_ID))

rank given by algorithm 3
4.6555644985420616e-10


In [93]:
def RRQR_3(A, TOL=None):
    m = len(A)                  #number of rows
    n = len(A[0])               #number of cols
    colnrm = np.zeros(n)
    for j in range(n):          #insert values into column norm vector
        colnrm[j] = np.linalg.norm(A[:,j], ord=2)
    
    
    for i in range(n):
        max_col = max(abs(colnrm[i:]))
        max_col_index = np.argmax(colnrm)
        A = Swap(A,j,max_col_index)
        colnrmm = Swap(colnrm, j, max_col_index)
        
    
    
    
    Q,R = scipy.qr(A)                           #get qr decomposition with pivoting from scipy                  +
    k = 1                                       #starting rank = 1
    r_1 = R[0,:]                                #starting slices of the QR decomp, not important just for reference
    r_11 = R[0,0]
    r_22 = R[1:,1:]
    q_1 = Q[:,0]
    q_2 = Q[:,1:]
    
    if TOL == None:                             #default tolerance if none is given
        TOL = 10e-5
    
    err = np.linalg.norm(r_22, ord=2)           #error is equal to norm of the part of the approximated matrix that is not used
    while err > TOL and k<min(len(A), len(A[0])):
        #print("current rank, error",k, err)                              #while error is greater than tolerance, increase
        k += 1                                  #increase rank
        
        r_1 = R[:k,:]                           #update slices
        r_11 = R[:k,:k]
        r_22 = R[k:,k:]
        q_1 = Q[:,:k]
        q_2 = Q[:,k:]
        
        if len(r_11[0]) == len(R[0]):           #if approximate array is max length of R, can't update error
            None
        else:
            err = np.linalg.norm(r_22, ord=2)
        
    #print("final error: ",np.linalg.norm(r_22, ord=2))
    
    return k

In [94]:
ap = apx_rank_k(100, 25,10)
res1 = RRQR_1(ap, TOL = 10e-10)
res3 = RRQR_3(ap, TOL = 10e-10)
print(res1, res3)

24 24


0.991
